# Glacier Gallery

## Import packages

In [ ]:
import panel as pn
import pandas as pd
import holoviews as hv
import geoviews as gv
from holoviews.plotting.links import Link
from holoviews.plotting.bokeh import LinkCallback
import geoviews.tile_sources as gts
from bokeh.models import HoverTool

hv.extension('bokeh', width=100)
pn.extension()
gv.extension('bokeh')

## Choose language for html document!

In [ ]:
language = 'en'  # 'en' or 'de'
file_name = f'gallery-app_{language}.html'

### Import text in different languages and selection of glacier with meta data

In [ ]:
# import all texts in the corresponding language from here:
from international import trads, supported_languages
from glacier_data import glaciers, links

### Function to prepare coordinates

In [ ]:
import math
from ast import literal_eval

def merc(Coords):
    lat = Coords[0]
    lon = Coords[1]

    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale    
    return (x, y)

## Load data

In [ ]:
df_all = pd.read_hdf('./data/rgi62_era5_itmix_df.h5', 'df')
df_all['rgi_id']= list(df_all.axes[0])
df_sel = df_all[['rgi_id', 'CenLon', 'CenLat', 'Lmax', 'GlacierType',
                 'TerminusType', 'GlacierStatus', 'IsTidewater',
                 'IsNominal' ]].copy()

## Select glaciers, add photos and build lists with information about them

In [ ]:
Glacier_No = glaciers # imported glaciers and meta data from Glaciers.py

# Create lists from glacier list above
glacier_list = []
glacier_names = []
gltype = []
glacier_pics =[]
glpic_pretext_source = []
glpic_cite = []
glpic_lic = []
glpic_wi = []
glpic_he = []
glpic_src = []
for gl_no in Glacier_No:
    glacier_list.append(gl_no[0])
    glacier_names.append(gl_no[1])
    gltype.append(gl_no[2])
    glacier_pics.append(gl_no[3])
    glpic_pretext_source.append(gl_no[4])
    glpic_cite.append(gl_no[5])
    glpic_lic.append (gl_no[6])
    glpic_wi.append(gl_no[8])
    glpic_he.append(gl_no[9])
    glpic_src.append(gl_no[10])

# create a new dataframe with the selected glaciers from the loaded data about glaciers 
df = df_sel.loc[df_sel['rgi_id'].isin(glacier_list)].copy()
df['name'] = glacier_names
df['Coords'] = list(zip(round(df.CenLat,2), round(df.CenLon,2)))
df['type'] = gltype
df['img'] = glacier_pics
df['pic_pretext'] = glpic_pretext_source
df['glpic_lic'] = glpic_lic
df['pic_cite'] = glpic_cite
df['pic_width'] = glpic_wi
df['pic_height'] = glpic_he
df['pic_src'] = glpic_src

## Define texts in according language

In [ ]:
title                 = '<div style="font-size:38px; color: #326a86; font-weight: bold;" >{}</div>'.format(trads['Title_hp'][language])
instruction           = '<div style="font-size:15px; color: #326a86; text-align: center;" >{}</div>'.format(trads['instruction'][language])
link_to_license_intro = '<div>{}</div>'.format(trads['link_to_license_intro'][language])
links_pic_src_intro   = '<div>{}</div>'.format(trads['links_pic_src_intro'][language])

links_to_license = links['links_license']
links_pic_src    = links['links_pics']

# translate "terminating" entry:
df.loc[df['TerminusType'] == 'Land-terminating', 'TerminusType']   = trads['TerminusType_land'][language]
df.loc[df['TerminusType'] == 'Marine-terminating', 'TerminusType'] = trads['TerminusType_marine'][language]

# translate "type" entry:
df.loc[df['type'] == 'Pure ice glacier', 'type']       = trads['type_pure_ice'][language]
df.loc[df['type'] == 'Calving glacier', 'type']        = trads['type_calving'][language]
df.loc[df['type'] == 'Debris covered glacier', 'type'] = trads['type_debris'][language]
df.loc[df['type'] == 'Ice cap', 'type']                = trads['type_ice_cap'][language]

# translate "pic_pretext":
df.loc[df['pic_pretext'] == 'Photo courtesy', 'pic_pretext'] = trads['pic_pretext'][language]

# define colors for dots of different glacier types
glacier_type_colors = {trads['type_pure_ice'][language]: '#56B4E9',
                       trads['type_calving'][language]: '#F0E442',
                       trads['type_debris'][language]: '#009E73',
                       trads['type_ice_cap'][language]: '#D55E00'}

## Build special datastructure with all information

In [ ]:
# prepare coordinates  
df.loc[:, 'coords_x'] = df['Coords'].apply(lambda x: merc(x)[0])
df.loc[:, 'coords_y'] = df['Coords'].apply(lambda x: merc(x)[1])

df.loc[:, 'LonDeg'] = df['CenLon']
df.loc[:, 'LatDeg'] = df['CenLat']

## Design for map with hover and webpage

In [ ]:
# background map
background_map = gts.tile_sources['CartoLight']

# define hover with image and values to display
TOOLTIPS =  """
    <div>
        <div>
            <img
                src="@img" 
                height="@pic_height" 
                alt="image loading..." 
                width="@pic_width"
                style=" margin: 2px 2px 0px 2px;"
                border="2">
            </img>
        </div>
        <div>
                <span style="font-size: 17px; font-weight: bold;">@name                </span> <br>
                <span style="font-size: 15px;                   ">@type                </span> <br>
                <span style="font-size: 15px;                   ">{}:              </span>
                <span style="font-size: 15px;                   ">@Lmax                </span>
                <span style="font-size: 15px;                   ">m                    </span> <br>
                <span style="font-size: 15px;                   ">@TerminusType        </span> <br>
                <span style="font-size: 15px;                   ">{}:            </span>
                <span style="font-size: 13px; color: #696;      ">(@LatDeg, @LonDeg)   </span> <br>
                <span style="font-size: 9px;                    ">@pic_pretext @pic_cite @glpic_lic </span>
        </div>
    </div>
            """.format(trads['tooltip_length'][language], trads['tooltip_loc'][language])
hover = HoverTool(tooltips=TOOLTIPS)

# glacier points, include all needed data (everything from hover info to text description) in vdims
glacier_points = gv.Points(df,
                           kdims=['CenLon', 'CenLat'],
                           vdims=['type', 'name', 'Lmax', 'TerminusType', 'pic_pretext', 'pic_cite',
                                  'glpic_lic', 'img', 'pic_height', 'pic_width', 'LatDeg', 'LonDeg']
                          ).opts(default_tools=['tap', 'wheel_zoom', 'reset', 'pan', hover],
                                 size=10,
                                 line_color='black',
                                 fill_alpha=0.9,
                                 fill_color='type',
                                 cmap=glacier_type_colors,
                                 xaxis=None,
                                 yaxis=None,
                                )

# put together the total figure
total_map = (glacier_points * background_map).opts(width=1000,
                                                   height=650,
                                                   legend_opts={'click_policy': 'none'},
                                                  )

# texts for different tabs
glacier_definition_text = hv.Div(f"<h2>{trads['Glacier_title'][language]}</h2>"
                                 f"{trads['Glacier_definition'][language]}<br><br>"
                                 f"<i>{trads['Glacier_text_source'][language]}</i>"
                                ).opts(sizing_mode='stretch_both')

pure_ice_text = hv.Div(f"<h2>{trads['type_pure_ice'][language]}</h2>"
                       f"{trads['pure_ice_text'][language]}<br><br>"
                       f"<i>{trads['pure_ice_source'][language]}</i>"
                      ).opts(sizing_mode='stretch_both')

calving_text = hv.Div(f"<h2>{trads['type_calving'][language]}</h2>"
                      f"{trads['calving_text'][language]}<br><br>"
                      f"<i>{trads['calving_source'][language]}</i>"
                     ).opts(sizing_mode='stretch_both')

debris_text = hv.Div(f"<h2>{trads['type_debris'][language]}</h2>"
                     f"{trads['debris_text'][language]}<br><br>"
                     f"<i>{trads['debris_source'][language]}</i>"
                    ).opts(sizing_mode='stretch_both')

ice_cap_text = hv.Div(f"<h2>{trads['type_ice_cap'][language]}</h2>"
                      f"{trads['ice_cap_text'][language]}<br><br>"
                      f"<i>{trads['ice_cap_source'][language]}</i>"
                     ).opts(sizing_mode='stretch_both')

# put together tab with texts
tabs = pn.Tabs((trads['tab_title_glacier_definition'][language],
                glacier_definition_text),
               (trads['tab_title_pure_ice'][language],
                pure_ice_text),
               (trads['tab_title_calving'][language],
                calving_text),
               (trads['tab_title_debris'][language],
                debris_text),
               (trads['tab_title_ice_cap'][language],
                ice_cap_text),
               width=400)


# this Div's is a hacky workaround to link the hv selection to a pn object
linking_div_hv = hv.Div(' ')
linking_div_pn = pn.panel(linking_div_hv, visible=False)

# first we link the plot to hv Div
class TextLinkhv(Link):
    _requires_target = True

class TextCallbackhv(LinkCallback):
    
    source_model = 'selected'
    source_handles = ['cds']
    on_source_changes = ['indices']

    target_model = 'plot'
    
    source_code = """
        var inds = source_selected.indices

        if (inds.length == 0) {
            target_plot.text = ' '
        } else {
            target_plot.text = source_cds.data['type'][inds[0]]
        }
    """

TextLinkhv.register_callback('bokeh', TextCallbackhv)
TextLinkhv(glacier_points, linking_div_hv)

# Now we link the pn Div to the tabs
code_pn_link = f"""
    if (source.text == ' ')
        target.active = 0
    if (source.text == '{trads['type_pure_ice'][language]}')
        target.active = 1
    if (source.text == '{trads['type_calving'][language]}')
        target.active = 2
    if (source.text == '{trads['type_debris'][language]}')
        target.active = 3
    if (source.text == '{trads['type_ice_cap'][language]}')
        target.active = 4
"""

linking_div_pn.jslink(tabs, code={'text': code_pn_link});

## Logos

In [ ]:
oggm_logo = ('<p style="margin-top: 0px;margin-bottom: 0px;"><a href="http://edu.oggm.org">'
             '<img src="https://raw.githubusercontent.com/zschirmeister/glacier-gallery/master/oggm_loupe.png" width=220>'
             '</a></p>')
hv_logo = ('<p style="margin-top: 0px;margin-bottom: 0px;"><a href="https://holoviz.org">'
           '<img src="https://holoviz.org/assets/holoviz-logo-stacked.svg" width=80>'
           '</a></p>')
fk_logo = ('<p style="margin-top: 0px;margin-bottom: 0px;"><a href="https://www.uibk.ac.at/foerderkreis1669/">'
           '<img src="https://raw.githubusercontent.com/zschirmeister/glacier-gallery/master/uibk_logo_narrow.png" width=140>'
           '</a></p>')

## Design application

In [ ]:
# Panel
# build header
header_height = 98
oggm_logo_pn = pn.Column(pn.Spacer(sizing_mode='stretch_height',
                                   margin=(0,0)),
                         pn.panel(oggm_logo,
                                  margin=(0, 0),
                                  height=70),
                        pn.Spacer(sizing_mode='stretch_height',
                                   margin=(0,0)),
                        height=header_height)

title_instruction_pn = pn.Column(
    pn.Row(pn.Spacer(sizing_mode='stretch_width', margin=(0, 0)),
           pn.panel(title, margin=(0, 0),
                   ),
           pn.Spacer(sizing_mode='stretch_width', margin=(0, 0)),
           sizing_mode='stretch_width'),
    pn.Row(pn.Spacer(sizing_mode='stretch_width', margin=(0, 0)),
           pn.panel(instruction, margin=(0, 0),
                    width=800,),
           pn.Spacer(sizing_mode='stretch_width', margin=(0, 0)),
           margin=(0, 0),
           sizing_mode='stretch_width'),
    margin=(0, 0),
    sizing_mode='stretch_width')

logos_pn = pn.Column(
    pn.Spacer(sizing_mode='stretch_height',
                                   margin=(0,0)),
    pn.Row(pn.Pane(fk_logo,
                   align='center',
                   height=85,
                   margin=(0, 0)),
           pn.Spacer(width=50, margin=(0, 0)),
           pn.Pane(hv_logo,
                   align='center',
                   height=85,
                   margin=(0, 0))
          ),
    pn.Spacer(sizing_mode='stretch_height',
                                   margin=(0,0)))
header = pn.Row(oggm_logo_pn, 
                pn.Spacer(width=10,
                          height=10,
                          margin=(0, 0)), 
                title_instruction_pn,
                pn.Spacer(width=10,
                          height=10,
                          margin=(0, 0)),
                logos_pn,
                pn.Spacer(width=30,
                          height=10,
                          margin=(0, 0)),
                margin=(0, 0),
                sizing_mode='stretch_width'
                )

links = pn.Row(pn.Column(links_pic_src_intro,
                         links_pic_src,),
               pn.Spacer(width=10),
               pn.Column(link_to_license_intro,
                         links_to_license,),
              )

# Put plot and text together and show it
app = pn.Column(header,
                pn.Spacer(height=2),
                pn.Row(total_map,
                       pn.Spacer(width=2),
                       tabs,
                       sizing_mode='stretch_both'),
                pn.Spacer(height=10),
                links,
                linking_div_pn,
                margin=(0, 0),
                sizing_mode='stretch_both')

#app.show()

In [ ]:
# save as a stand-alone HTML
app.save(file_name, 'Glacier Gallery')